In [1]:
import os
import time
import pyodbc
import textwrap
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from IPython.core.display import HTML

from dotenv import load_dotenv
from openai import AzureOpenAI
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain

# Warning control
import warnings
warnings.filterwarnings("ignore")

In [2]:
load_dotenv('.env.studiomac', override=True)

NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE')

AZURE_RESOURCE_NAME=os.getenv('AZURE_RESOURCE_NAME')
AZURE_ENDPOINT=os.getenv('AZURE_ENDPOINT')
AZURE_API_KEY1=os.getenv('AZURE_API_KEY1')
AZURE_API_KEY2=os.getenv('AZURE_API_KEY2')
AZURE_EMBED_DEPLOYMENT=os.getenv('AZURE_EMBED_DEPLOYMENT')
AZURE_CHATGPT_DEPLOYMENT=os.getenv('AZURE_CHATGPT_DEPLOYMENT')
AZURE_OPENAI_API_VERSION=os.getenv('AZURE_OPENAI_API_VERSION')

SR_DATABASE=os.getenv('SR_DATABASE')
SR_SERVER=os.getenv('SR_SERVER')
SR_USERNAME=os.getenv('SR_USERNAME')
SR_PASSWORD=os.getenv('SR_PASSWORD')
SR_DRIVER=os.getenv('SR_DRIVER')

## Establish relationships

In [3]:
HTML('<img src=".\imgs\kg1.png" alt="kg Image" width="500"/>')

In [4]:
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

In [5]:
# acquire a group of nodes with a thread id
result = kg.query("""
                MATCH (t:Thread), (u:User), (mod:Model), (msg:Message)
                WHERE t.ThreadId = u.ThreadId AND t.ThreadId = mod.ThreadId AND t.ThreadId = msg.source
                WITH t,u,mod, msg {.source, .MsgId} as msg_info
                RETURN t,u,mod, collect(msg_info) LIMIT 1
                """)
print(textwrap.fill(str(result),150))

[{'t': {'Category': 'Software', 'ThreadId': '20091207_0004', 'CreateDate': '2009-12-07T23:00:06.053000', 'Subject': 'Pass-through mode'}, 'u':
{'country': '', 'ThreadId': '20091207_0004', 'UserId': '20091207_0004-Hades', 'userIP': '77.78.151.5', 'username': 'Hades'}, 'mod': {'HMI_Model':
'MT6070iH', 'SerialNo': '09110371', 'PLC_Connection': 'RS485 4W', 'ThreadId': '20091207_0004', 'HMI_COMport': '   ', 'Driver': '', 'OS_Version': '',
'PLC_Model': 'Mitsubishi FX1N', 'ModelId': '20091207_0004-MT6070iH-Mitsubishi FX1N', 'Firmware_Version': 'MT8xxx(S3C) build 20091002'},
'collect(msg_info)': [{'MsgId': '20091207_0004_1', 'source': '20091207_0004'}, {'MsgId': '20091207_0004_2', 'source': '20091207_0004'}]}]


- Given a source and order by MsgId

In [6]:
message_nodes = kg.query("""
                        MATCH (msg:Message {source:"20240630_0001"})
                        RETURN msg {.MessageOrder,.source,.MsgId,.PostDate,.MessageText,.ReplyText,.Replier,.ReplyDate,.ReplyFile}
                        ORDER BY msg.MsgId
                        """)
for node in message_nodes:
    print(node)

{'msg': {'ReplyFile': None, 'MsgId': '20240630_0001_1', 'Replier': 'angushuang', 'PostDate': '2024-06-30T07:32:36.897000', 'source': '20240630_0001', 'MessageOrder': 1, 'ReplyText': '&nbsp;Hi Petar,<br />\r\n<br />\r\nWhat would you do after the memory reach the limit, 30000 record?<br />\r\nIn fac, HMI can’t precisely prompt you it saves 30,000 records.<br />\r\n<br />\r\nBest regards', 'ReplyDate': '2024-07-01T01:26:15.223000', 'MessageText': '<p>Dear Sir/Madam,</p><p>I have a task to log 30000 event logs to USB drive and after 30000 limit is reached, first record to automatically delete and to continue to operate in FIFO (First In First Out) mode.</p><p>Since the limit is 10000 logs to HMI memory, is it feasible to log more than 10000 to USB drive and where I can specify exact maximum number of logs which in this case is 30000?<br></p><p>It will be of great help if you have an example of how to do handle this situation.</p><br><p>Thank you in advance,</p><p>Petar<br></p>'}}
{'msg': 

### Add a NEXT relationship between subsequent chunks
- Use the `apoc.nodes.link` function from Neo4j to link ordered list of `Chunk` nodes with a `NEXT` relationship
- Do this for just the given threadId `20240630_0001` section to start

In [7]:
cypher = """
        MATCH (msg:Message)  
        WHERE msg.source = $source  
        WITH msg  
        ORDER BY msg.MsgId  
        WITH collect(msg) as msg_list  
        CALL apoc.nodes.link(  
            msg_list,   
            "NEXT",
            {avoidDuplicates: true}  
        )  
        RETURN size(msg_list) as size_info
        """
kg.query(cypher,
         params={"source":"20240630_0001"})

[{'size_info': 5}]

In [8]:
# search the paattern
cypher = """
  MATCH (ms:Message)-[:NEXT]->(me:Message) 
  RETURN ms.MsgId as ms, me.MsgId as me
  LIMIT 10
  """
kg.query(cypher)

[{'ms': '20240630_0001_1', 'me': '20240630_0001_2'},
 {'ms': '20240630_0001_2', 'me': '20240630_0001_3'},
 {'ms': '20240630_0001_3', 'me': '20240630_0001_4'},
 {'ms': '20240630_0001_4', 'me': '20240630_0001_5'}]

In [9]:
# get next message for a given message id
cypher = """
  MATCH (firstMsg:Message)-[:NEXT]->(nextMsg:Message)
    WHERE firstMsg.MsgId = $msgIdParam
  RETURN nextMsg.MsgId as msgId, nextMsg.MessageText as text
"""

next_chunk_info = kg.query(cypher, params={
    'msgIdParam': "20240630_0001_2"
})[0]

next_chunk_info

{'msgId': '20240630_0001_3',
 'text': '<p>Hello,</p><p><br></p><p>I understand but lets analyze different approach to this task.</p><br><p>If I create a database file like barcode.db on USB disk, and use for example HMI cMT3072X instead of cMT2108X2 which does not support SQL server and queries, can I record more than 10000 records to this database?<br></p>'}

In [10]:
kg.refresh_schema()
print(kg.schema)

Node properties:
Thread {ThreadId: STRING, Subject: STRING, CreateDate: STRING, Category: STRING}
User {ThreadId: STRING, UserId: STRING, username: STRING, country: STRING, userIP: STRING}
Model {ThreadId: STRING, ModelId: STRING, HMI_Model: STRING, HMI_COMport: STRING, Firmware_Version: STRING, OS_Version: STRING, SerialNo: STRING, PLC_Model: STRING, Driver: STRING, PLC_Connection: STRING}
Message {MsgId: STRING, MessageOrder: INTEGER, source: STRING, PostDate: STRING, MessageText: STRING, ReplyText: STRING, Replier: STRING, ReplyDate: STRING, textEmbedding: LIST, ReplyFile: STRING}
Relationship properties:

The relationships:



- Loop through threads and create relationships for all messages

In [11]:
# step1: acquire all threadId
threadId_list = kg.query("""MATCH (t:Thread)
                         RETURN t.ThreadId AS threadId
                         """)
# step2: scan each id and create next relationshipt between Message Node
cypher = """
        MATCH (msg:Message)  
        WHERE msg.source = $source  
        WITH msg  
        ORDER BY msg.MsgId  
        WITH collect(msg) as msg_list  
        CALL apoc.nodes.link(  
            msg_list,   
            "NEXT",
            {avoidDuplicates: true}  
        )  
        RETURN size(msg_list) as size_info 
        """
for item in tqdm(threadId_list, desc="Processing threadId",unit="threads"):
    source = item['threadId']
    size_info = kg.query(cypher, 
                         params={"source":source})

Processing threadId: 100%|██████████| 9963/9963 [40:14<00:00,  4.13threads/s]


- Retrieve the longest path for NEXT

In [12]:
cypher = """
        MATCH window=
            (:Message)-[:NEXT*0..10]->(m:Message)-[:NEXT*0..10]->(:Message)
          WHERE m.MsgId = $firstMsgIdParam
        WITH window as longestChunkWindow 
            ORDER BY length(window) DESC LIMIT 1
        RETURN length(longestChunkWindow) as max_len, longestChunkWindow
        """

result = kg.query(cypher,
                  params={'firstMsgIdParam': "20240630_0001_1"})
print("max_len: ",result[0]["max_len"])
print(result[0]['longestChunkWindow'][0])
print(result[0]['longestChunkWindow'][1])
print(result[0]['longestChunkWindow'][2])

max_len:  4
{'Replier': 'angushuang', 'textEmbedding': [-0.04360667243599892, 0.03185044229030609, -0.013473344035446644, 0.014072336256504059, 0.004959660116583109, 0.012203479185700417, -0.019774748012423515, 0.04402197524905205, -0.01251495536416769, 0.02594037912786007, 0.019087903201580048, -0.030061449855566025, -0.03256923332810402, 0.006768618244677782, -0.0039273956790566444, -0.0036798121873289347, -0.033096347004175186, -0.013641061261296272, 0.009823481552302837, -0.004288787953555584, 0.048750024288892746, 0.0009279396035708487, -0.00932831410318613, 0.05031539127230644, -5.964969386695884e-05, 0.035236746072769165, -0.031131649389863014, -0.0110454261302948, 0.007287745364010334, -0.010518312454223633, -0.01395253837108612, 0.03833553567528725, 0.016803743317723274, -0.02969406731426716, -0.03175460174679756, -0.035077016800642014, 0.07532932609319687, 0.035460371524095535, -0.013249719515442848, -0.001209965324960649, 0.0008785226964391768, 0.01717112585902214, -0.010973

### Connect Message to their parent form with a PART_OF relationship

In [13]:
cypher = """
  MATCH (m:Message), (t:Thread)
    WHERE m.source = t.ThreadId
  MERGE (m)-[newRelationship:PART_OF]->(t)
  RETURN count(newRelationship) as count
"""

kg.query(cypher)

[{'count': 20973}]

### Create OPEN relationship from User to Thread

In [14]:
cypher = """
  MATCH (u:User), (t:Thread)
    WHERE u.ThreadId = t.ThreadId
  MERGE (u)-[newRelationship:OPEN]->(t)
  RETURN count(newRelationship) as count
"""

kg.query(cypher)

[{'count': 9963}]

### Create USE relationship from User to Model

In [15]:
cypher = """
  MATCH (u:User), (m:Model)
    WHERE u.ThreadId = m.ThreadId
  MERGE (u)-[newRelationship:USE]->(m)
  RETURN count(newRelationship) as count
"""

kg.query(cypher)

[{'count': 9963}]

In [16]:
kg.refresh_schema()
print(kg.schema)

Node properties:
Thread {ThreadId: STRING, Subject: STRING, CreateDate: STRING, Category: STRING}
User {ThreadId: STRING, UserId: STRING, username: STRING, country: STRING, userIP: STRING}
Model {ThreadId: STRING, ModelId: STRING, HMI_Model: STRING, HMI_COMport: STRING, Firmware_Version: STRING, OS_Version: STRING, SerialNo: STRING, PLC_Model: STRING, Driver: STRING, PLC_Connection: STRING}
Message {MsgId: STRING, MessageOrder: INTEGER, source: STRING, PostDate: STRING, MessageText: STRING, ReplyText: STRING, Replier: STRING, ReplyDate: STRING, textEmbedding: LIST, ReplyFile: STRING}
Relationship properties:

The relationships:
(:User)-[:OPEN]->(:Thread)
(:User)-[:USE]->(:Model)
(:Message)-[:PART_OF]->(:Thread)
(:Message)-[:NEXT]->(:Message)


In [ ]:
# search for a window for a given threadid
cypher = """MATCH window = 
            (node:Message {source:"20240630_0001"})-[:PART_OF]->(thread:Thread)<-[:OPEN]-(user:User)-[:USE]->(model:Model)
            RETURN window"""

### Customize the results of the similarity search using Cypher
- Extend the vector store definition to accept a Cypher query
- The Cypher query takes the results of the vector similarity search and then modifies them in some way
- Start with a simple query that just returns some extra text along with the search results

In [17]:
# Create Embedding
Embeddings = AzureOpenAIEmbeddings(
    azure_deployment=AZURE_EMBED_DEPLOYMENT,
    openai_api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_ENDPOINT,
    api_key=AZURE_API_KEY1,
)


VECTOR_INDEX_NAME = "form_user_message"
VECTOR_NODE_LABEL = "Message"
VECTOR_SOURCE_PROPERTY1 = 'MessageText'
VECTOR_SOURCE_PROPERTY2 = "RepleyText"
VECTOR_EMBEDDING_PROPERTY = 'textEmbedding'

# modify the retrieval extra text here then run the entire cell
retrieval_query_extra_text = """
WITH node, score
RETURN "Question:" + "\n" + node.MessageText + "\n" + "Answer:" + "\n" + node.ReplyText as text,
    score,
    node {.source, .MsgId} AS metadata
"""

vector_store_extra_text = Neo4jVector.from_existing_index(
    embedding=Embeddings,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database="neo4j",
    index_name=VECTOR_INDEX_NAME,
    text_node_property=VECTOR_SOURCE_PROPERTY1,
    retrieval_query=retrieval_query_extra_text, # NEW !!!
)

# Create a retriever from the vector store
retriever_extra_text = vector_store_extra_text.as_retriever(search_kwargs={"k": 10})


# Create a chatbot Question & Answer chain from the retriever
llm = AzureChatOpenAI(
            openai_api_type="azure",
            openai_api_version=AZURE_OPENAI_API_VERSION,
            openai_api_key=AZURE_API_KEY1,
            azure_endpoint=AZURE_ENDPOINT,
            max_tokens=4096,
            temperature=0.7,
            top_p=0.95,
            frequency_penalty=0.0,
            presence_penalty=0.0,
            azure_deployment=AZURE_CHATGPT_DEPLOYMENT,
            # streaming=True,
            max_retries=3,
            timeout=60,
        )
chain_extra_text = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever_extra_text
)


# Use Original API to chat with gpt
def chat_completion_openai_history(openai_history_messages:list=[], 
                                   model=AZURE_CHATGPT_DEPLOYMENT):    
    # define return fmt
    return_json={
                'prompt_tokens': 0,
                'completion_tokens':0,
                'total_tokens': 0,
                'status':"fail",
                'selected_model':model,
                'replied_message':"",
                'error_reason':""
            }
    try:
        client = AzureOpenAI(
                            api_key = AZURE_API_KEY1, 
                            api_version = AZURE_OPENAI_API_VERSION,
                            azure_endpoint = AZURE_ENDPOINT
                        )
        chat_completion = client.chat.completions.create(
                model=model,
                temperature=0.7,
                max_tokens=4096,
                top_p=0.95,
                frequency_penalty=0,
                presence_penalty=0,
                stop=None,
                messages=[{"role":"system","content":"You are an AI assistant that helps people find information."}] + openai_history_messages
            )
        return_json['prompt_tokens'] = chat_completion.usage.prompt_tokens
        return_json['completion_tokens'] = chat_completion.usage.completion_tokens
        return_json['total_tokens'] = chat_completion.usage.total_tokens
        return_json['replied_message'] = chat_completion.choices[0].message.content
        return_json['prompt_tokens'] = chat_completion.usage.prompt_tokens
        return_json['status'] = "success"
        # print(chat_completion)
    except Exception as e:
        error_reason = f"[OPENAI ERROR]{str(e)}"
        return_json['error_reason'] = error_reason
        print(error_reason)
    return return_json


In [18]:
# search by retriever
result = retriever_extra_text.invoke("I can not open my file with your easybuilder software.")
result

[Document(metadata={'MsgId': '20100423_0001_1', 'source': '20100423_0001'}, page_content='Question:\nI can not open my file with your easybuilder software.&nbsp; I have tried several differnet version and i always receive and error saying &quot;Must use a newer tool to open&quot;.<br />\r\n<br />\r\nHow do i find out what version i need, unable to get ahold of oem who supplied.<br />\nAnswer:\n<p>Which EB8000 version you used before? have you open your project with EB8000v4.00 or v4.02beta version?&nbsp;<br />\r\n&nbsp;</p>'),
 Document(metadata={'MsgId': '20190109_0001_4', 'source': '20190109_0001'}, page_content='Question:\n<p>Okay thanks,</p><br><p>Can you also look to the easybuilder file. I cannot open this file anymore.<br></p>\nAnswer:\nDear Sir,<br />\r\n<br />\r\nWhat version did you use to make this project?<br />\r\nBefore it can’t open with V6.02.01, what did you do?<br />\r\n<br />\r\nBest Regards,<br />\r\nFinn<br />\r\n<br type="_moz" />'),
 Document(metadata={'MsgId': '

### Expand context around a Message using a window
- Use retriever_extra_text to try llm response
- This retriever is viewed as windowless retriever (because we just use QA as our reference)

In [19]:
# For windowless retriever
question = "I can not open my file with your easybuilder software. please help me and give me some advice"
result = retriever_extra_text.invoke(question)
# print(result)

# create a stuff prompt
stuff_prompt = ""
for res in result:
    print(res.page_content)
    stuff_prompt += res.page_content + "\n"


# create a prompt for openai api call
prompt = f"""
Here are some QA reference for you to refer to:
{stuff_prompt}
The user's question is:
Question: {question}
Please answer the question based on the above reference. If the reference is not enough to reply, 
please say "The reference provided is insufficient to answer the question."
Replied Message: <your summary>
"""

print(prompt)

# create openai user messages and call openai api
openai_history_messages = [{"role": "user", "content": prompt}]
return_json = chat_completion_openai_history(openai_history_messages)
print(return_json['replied_message'])


Question:
I can not open my file with your easybuilder software.&nbsp; I have tried several differnet version and i always receive and error saying &quot;Must use a newer tool to open&quot;.<br />
<br />
How do i find out what version i need, unable to get ahold of oem who supplied.<br />
Answer:
<p>Which EB8000 version you used before? have you open your project with EB8000v4.00 or v4.02beta version?&nbsp;<br />
&nbsp;</p>
Question:
can you send me the link what software is good to open it<br />
i got easybuilder 500<br />
please do let me know<br />
Answer:
Dear Sir<br />
<br />
Please refer to the link for downloading EB500 V2.74. Thanks<br />
ftp://ftp.weintek.com/MT500/English/EB500V274_080130_enU.zip<br type="_moz" />
Question:
<p>A message keep on appearing if I open older .emtp files in the Easybuilder program</p><br><p>I need urgent help please</p><br>
Answer:
Dear sir,<br />
<br />
Could you please attach a picture for me to check the error message?<br />
<br />
Thanks
Questi

- Next, define a window retrieval query to get consecutive chunks

In [20]:
retrieval_query_window = """
MATCH window=
    (:Message)-[:NEXT*0..1]->(node)-[:NEXT*0..1]->(:Message),
    (node)-[:PART_OF]->(thread:Thread)<-[:OPEN]-(user:User)-[:USE]->(model:Model)
WITH node, score, window, thread, user, model
    ORDER BY node.source, length(window) DESC
WITH collect(window)[0] as longestWindow, node, score, thread, user, model
WITH nodes(longestWindow) as chunkList, node, score, thread, user, model
  UNWIND chunkList as chunkRows
WITH collect("Question:" + "\n" + chunkRows.MessageText + "\n" + "Answer:" + "\n" + chunkRows.ReplyText) as QA_RefList, node, score, thread, user, model
RETURN "Form Id:" + thread.ThreadId + "\n" + "Form Subject:" + thread.Subject + "\n" + "HMI Model:"+ model.HMI_Model + "\n" + "Q & A process:"+ "\n" + apoc.text.join(QA_RefList, " \n ") as text,
    score,
    node {.source, .MsgId} AS metadata
"""


# retrieval_query_window = """
# MATCH window=
#     (:Message)-[:NEXT*0..1]->(node)-[:NEXT*0..1]->(:Message)
# WITH node, score, window
#     ORDER BY node.MsgId, length(window) DESC
# WITH collect(window)[0] as longestWindow, node, score
# WITH nodes(longestWindow) as chunkList, node, score
#   UNWIND chunkList as chunkRows
# WITH collect("Question:" + "\n" + chunkRows.MessageText + "\n" + "Answer:" + "\n" + chunkRows.ReplyText) as QA_RefList, node, score
# RETURN apoc.text.join(QA_RefList, " \n ") as text,
#     score,
#     node {.source, .MsgId} AS metadata
# """



# set up window retriever
vector_store_window = Neo4jVector.from_existing_index(
    embedding=Embeddings,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database="neo4j",
    index_name=VECTOR_INDEX_NAME,
    text_node_property=VECTOR_SOURCE_PROPERTY1,
    retrieval_query=retrieval_query_window, # NEW !!!
)

# Create a retriever from the vector store
retriever_window = vector_store_window.as_retriever(search_kwargs={"k": 10})

- Compare the two responses

In [21]:
# search by window retriever
question = "I can not open my file with your easybuilder software. please help me and give me some advice"
result = retriever_window.invoke(question)
result

[Document(metadata={'MsgId': '20100423_0001_1', 'source': '20100423_0001'}, page_content='Form Id:20100423_0001\nForm Subject:Can not open mtp file\nHMI Model:MT6070iH\nQ & A process:\nQuestion:\nI can not open my file with your easybuilder software.&nbsp; I have tried several differnet version and i always receive and error saying &quot;Must use a newer tool to open&quot;.<br />\r\n<br />\r\nHow do i find out what version i need, unable to get ahold of oem who supplied.<br />\nAnswer:\n<p>Which EB8000 version you used before? have you open your project with EB8000v4.00 or v4.02beta version?&nbsp;<br />\r\n&nbsp;</p> \n Question:\nI was told it was developed in 3.4 and i tried to open it in that version and receive the below error<br />\r\n<br />\r\n&quot;An unknown error occurred while accessing an unnamed file.&quot;\nAnswer:\nCould you please send your project for testing? <br />\r\nAre you using V3.40,&nbsp;V3.42&nbsp;or V3.44 to develop the project?'),
 Document(metadata={'MsgId':

In [22]:
# search by windowless retriever
result = retriever_extra_text.invoke(question)
result

[Document(metadata={'MsgId': '20100423_0001_1', 'source': '20100423_0001'}, page_content='Question:\nI can not open my file with your easybuilder software.&nbsp; I have tried several differnet version and i always receive and error saying &quot;Must use a newer tool to open&quot;.<br />\r\n<br />\r\nHow do i find out what version i need, unable to get ahold of oem who supplied.<br />\nAnswer:\n<p>Which EB8000 version you used before? have you open your project with EB8000v4.00 or v4.02beta version?&nbsp;<br />\r\n&nbsp;</p>'),
 Document(metadata={'MsgId': '20120420_0004_3', 'source': '20120420_0004'}, page_content='Question:\ncan you send me the link what software is good to open it<br />\r\ni got easybuilder 500<br />\r\nplease do let me know<br />\nAnswer:\nDear Sir<br />\r\n<br />\r\nPlease refer to the link for downloading EB500 V2.74. Thanks<br />\r\nftp://ftp.weintek.com/MT500/English/EB500V274_080130_enU.zip<br type="_moz" />'),
 Document(metadata={'MsgId': '20190426_0003_1', 's

In [24]:
def create_stuff_prompt(result):
    stuff_prompt = ""
    for res in result:
        # print(res.page_content)
        stuff_prompt += res.page_content + "\n"
    return stuff_prompt


def create_llm_prompt(stuff_prompt, question):
    prompt = f"""
    Here are some QA reference for you to refer to:
    {stuff_prompt}
    The user's question is:
    Question: {question}
    Please answer the question based on the above reference. If the reference is not enough to reply, 
    please say "The reference provided is insufficient to answer the question."
    Replied Message: <your summary>
    """
    return prompt

In [25]:
# For window retriever
question = "I can not open my file with your easybuilder software. please help me and give me some advice"
result = retriever_window.invoke(question)
stuff_prompt = create_stuff_prompt(result)
llm_prompt = create_llm_prompt(stuff_prompt, question)
print(llm_prompt)
print("================================"*2)

# create openai user messages and call openai api
openai_history_messages = [{"role": "user", "content": llm_prompt}]
return_json = chat_completion_openai_history(openai_history_messages)
print(return_json['replied_message'])



    Here are some QA reference for you to refer to:
    Form Id:20100423_0001
Form Subject:Can not open mtp file
HMI Model:MT6070iH
Q & A process:
Question:
I can not open my file with your easybuilder software.&nbsp; I have tried several differnet version and i always receive and error saying &quot;Must use a newer tool to open&quot;.<br />
<br />
How do i find out what version i need, unable to get ahold of oem who supplied.<br />
Answer:
<p>Which EB8000 version you used before? have you open your project with EB8000v4.00 or v4.02beta version?&nbsp;<br />
&nbsp;</p> 
 Question:
I was told it was developed in 3.4 and i tried to open it in that version and receive the below error<br />
<br />
&quot;An unknown error occurred while accessing an unnamed file.&quot;
Answer:
Could you please send your project for testing? <br />
Are you using V3.40,&nbsp;V3.42&nbsp;or V3.44 to develop the project?
Form Id:20111113_0001
Form Subject:Blank Screen - EasyView
HMI Model:Other
Q & A process:
Que

In [26]:
# For windowless retriever
question = "I can not open my file with your easybuilder software. please help me and give me some advice"
result = retriever_extra_text.invoke(question)
stuff_prompt = create_stuff_prompt(result)
llm_prompt = create_llm_prompt(stuff_prompt, question)
print(llm_prompt)
print("================================"*2)

# create openai user messages and call openai api
openai_history_messages = [{"role": "user", "content": llm_prompt}]
return_json = chat_completion_openai_history(openai_history_messages)
print(return_json['replied_message'])


    Here are some QA reference for you to refer to:
    Question:
I can not open my file with your easybuilder software.&nbsp; I have tried several differnet version and i always receive and error saying &quot;Must use a newer tool to open&quot;.<br />
<br />
How do i find out what version i need, unable to get ahold of oem who supplied.<br />
Answer:
<p>Which EB8000 version you used before? have you open your project with EB8000v4.00 or v4.02beta version?&nbsp;<br />
&nbsp;</p>
Question:
can you send me the link what software is good to open it<br />
i got easybuilder 500<br />
please do let me know<br />
Answer:
Dear Sir<br />
<br />
Please refer to the link for downloading EB500 V2.74. Thanks<br />
ftp://ftp.weintek.com/MT500/English/EB500V274_080130_enU.zip<br type="_moz" />
Question:
<p>A message keep on appearing if I open older .emtp files in the Easybuilder program</p><br><p>I need urgent help please</p><br>
Answer:
Dear sir,<br />
<br />
Could you please attach a picture for 